In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from summaryScraper import SummaryScraper
from selenium.common.exceptions import WebDriverException

In [ ]:
scraper = SummaryScraper()
for i in range(2005, 2007):
    scraper.etl(start=i)
    time.sleep(120)